In [ ]:
# Minimal helper: sftp.send_file(local_path, remote_path)
# - Reads mkndaq.yml
# - Instantiates SFTPClient from your sftp.py
# - Ensures remote folder exists
# - Uploads ONE file to the SFTP server
from pathlib import Path
from typing import Union
from pathlib import PurePosixPath

import yaml
import paramiko

from mkndaq.utils.sftp import SFTPClient

def _load_cfg(path: Union[str, Path] = "dist/mkndaq.yml") -> dict:
    with open(path, "r", encoding="utf-8") as fh:
        return yaml.safe_load(fh) or {}

def _normalize_remote_path(base: PurePosixPath, p: Union[str, PurePosixPath]) -> PurePosixPath:
    rp = PurePosixPath(str(p).replace("\\", "/"))
    # If the user passes a relative path, make it relative to the YAML's remote base
    return rp if str(rp).startswith("/") else (base / rp)

def send_file(
    local_path: Union[str, Path], 
    remote_path: Union[str, PurePosixPath] = 'ne300', 
    config_path: Union[str, Path] = "dist/mkndaq.yml") -> str:
    """
    Upload a single file via SFTP using mkndaq.yml settings.

    Parameters
    ----------
    local_path : str | Path
        Local file to upload.
    remote_path : str | PurePosixPath
        Destination path on the server. If relative, it is joined to the YAML's
        base 'sftp.remote' (and instrument remote if you include it in the path).
    config_path : str | Path
        Path to mkndaq.yml (default: "mkndaq.yml").

    Returns
    -------
    str : The final remote path that was written (POSIX).
    """
    lp = Path(local_path)
    if not lp.is_file():
        raise FileNotFoundError(f"Local file not found: {lp}")

    cfg = _load_cfg(config_path)

    # Build client from your sftp.py (reads keys/host/user and base paths)
    client = SFTPClient(cfg)  # name left default; remote base is cfg['sftp']['remote']
    rp = _normalize_remote_path(client.remote_path, remote_path)

    # Ensure destination folder exists using sftp.py helper
    client.setup_remote_path(rp.parent)

    # Upload using same credentials as SFTPClient
    with paramiko.SSHClient() as ssh:
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(hostname=client.host, username=client.usr, pkey=client.key)
        with ssh.open_sftp() as sftp_cli:
            sftp_cli.put(lp.as_posix(), rp.as_posix())

    return rp.as_posix()

In [ ]:
from pathlib import Path

local = Path(r"C:/users/mkn/Documents/staging/ne300/ne300-pickle.zip")
# Absolute remote path:
remote = f"ne300/{local.name}"

# Upload
written = send_file(local, remote)
print("Uploaded to:", written)